In [101]:
import pandas as pd
import yaml
import numpy as np

In [102]:
with open('bodies.yaml', 'r') as f:
    raw_planet_data = yaml.load(f)

In [103]:
earth = {'mass':5.972e24,
         'radius': 6.371e6
        }
sun = {'mass':1.989e30,
       'radius': 695.8e6
      }

In [104]:
columns = ['mass', 'period', 'radius', 'semi-major', 'eccentricity']

planet_data = pd.DataFrame(columns=columns)
star_data = pd.DataFrame(columns=columns)

for d in raw_planet_data.keys():
    current_dict = raw_planet_data[d]
    for c in current_dict.keys():
        if current_dict[c] == 'NA':
            current_dict[c] = np.nan
    if d == 'sun':
        pop_columns = [[float(x) for x in [ current_dict['mass'], np.nan, current_dict['radius'],np.nan,np.nan]]]
        df = pd.DataFrame(pop_columns,columns=columns)
        star_data = star_data.append(df)
        continue
    pop_columns = [[float(x) for x in [ current_dict['mass'], current_dict['period'], current_dict['radius'], current_dict['semi-major'], current_dict['eccentricity']]]]        
    df = pd.DataFrame(pop_columns,columns=columns)
    planet_data = planet_data.append(df, ignore_index=True)

In [105]:
planet_data['mass'] *= earth['mass']
planet_data['radius'] *= earth['radius']
planet_data['semi-major'] *= (10e-3 * 1.496e11)
planet_data['period'] *= 24*60*60

star_data['mass'] *= sun['mass']
star_data['radius'] *= sun['radius']

In [106]:
bodies = planet_data.T
bodies['star'] = star_data.T[0]
bodies

,0,1,2,3,4,5,6,star
mass,8.241360e+24,5.076200e+24,8.002480e+24,3.702640e+24,4.060960e+24,NaN,2.448520e+24,1.595178e+29
period,2.092455e+05,1.305392e+05,1.067294e+06,5.270067e+05,7.954580e+05,1.728000e+06,3.498863e+05,NaN
radius,6.727776e+06,6.918906e+06,7.180117e+06,5.848578e+06,6.657695e+06,4.810105e+06,4.918412e+06,8.140860e+07
semi-major,2.275416e+10,1.662056e+10,6.746960e+10,4.214232e+10,5.550160e+10,9.424800e+10,3.207424e+10,NaN
eccentricity,8.300000e-02,8.100000e-02,6.100000e-02,8.500000e-02,6.300000e-02,NaN,7.000000e-02,NaN
